# M1M3 cell learning
Craig Lage - 14-Apr-23 \
The 17 tons of mirror are supported by 156 pneumatic actuators where 44 are single-axis and provide support only on the axial direction, 100 are dual-axis providing support in the axial and lateral direction, and 12 are dual-axis providing support in the axial and cross lateral directions. \
Positioning is provided by 6 hard points in a hexapod configuration which moves the mirror to a fixed operational position that shall be maintained during telescope operations. The remaining optical elements will be moved relative to this position in order to align the telescope optics. Support and optical figure correction is provided by 112 dual axis and 44 single axis pneumatic actuators. 

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import pickle as pkl
from astropy.time import Time, TimeDelta
import lsst.ts.cRIOpy.M1M3FATable as M1M3FATable
from scipy.interpolate import UnivariateSpline
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('summit_efd')
FATABLE = M1M3FATable.FATABLE

In [ ]:
# Times of bump test
start = Time("2023-04-17T10:00:00", scale='utc')
end = Time("2023-04-17T11:15:00", scale='utc')

In [ ]:
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*", start, end)

In [ ]:
len(bumps)

## Given an actuator ID, this plots the bump applied forces

In [ ]:
async def plotBumpTestAppliedForces(fig, bumps, id):
    thisBump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"zForces{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"yForces{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"xForces{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    plt.subplots_adjust(wspace=0.3)
    plt.subplot(1,2,1)
    plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
    plotEnd = plotStart + 14.0 #thisBump[thisBump[primaryBump]==5]['timestamp'].values[0] + 2.0
    start = Time(plotStart, format='unix_tai', scale='tai')
    end = Time(plotEnd, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.appliedForces", [primaryForce, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    plotStart -= t0
    plotEnd -= t0
    plt.title(f"Primary - Z - ID:{id}")
    plt.plot(times, forces[primaryForce].values)
    plt.text(1.0, 350.0, "lsst.sal.MTM1M3.appliedForces."+primaryForce, color='g')

    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (nt)")
    plt.subplot(1,2,2)
    if secondaryName is not None:
        plt.title(f"Secondary - {secondaryName} - ID:{id}")
        plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
        plotEnd = plotStart + 14.0 #thisBump[thisBump[secondaryBump]==5]['timestamp'].values[0] + 2.0
        start = Time(plotStart, format='unix_tai', scale='tai')
        end = Time(plotEnd, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.appliedForces", [secondaryForce, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        plotStart -= t0
        plotEnd -= t0
        plt.plot(times, forces[secondaryForce].values)
        plt.text(1.0, 350.0, "lsst.sal.MTM1M3.appliedForces."+secondaryForce, color='g')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (nt)")
    else:
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
    return

In [ ]:
fig = plt.figure(figsize=(10,5))
await plotBumpTestAppliedForces(fig, bumps, 227)

## Given an actuator ID, this plots the bump test result

In [ ]:
async def plotBumpTestResults(fig, bumps, id):
    thisBump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"zForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        if FATABLE[index][M1M3FATable.FATABLE_ORIENTATION] in ['+Y', '-Y']:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"yForce{FATABLE[index][M1M3FATable.FATABLE_YINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
        else:
            secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
            secondaryForce = f"xForce{FATABLE[index][M1M3FATable.FATABLE_XINDEX]}"
            secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    plt.subplots_adjust(wspace=0.3)
    plt.subplot(1,2,1)
    plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
    plotEnd = plotStart + 14.0 #thisBump[thisBump[primaryBump]==5]['timestamp'].values[0] + 2.0
    start = Time(plotStart, format='unix_tai', scale='tai')
    end = Time(plotEnd, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    plotStart -= t0
    plotEnd -= t0
    plt.title(f"Primary - Z - ID:{id}")
    plt.plot(times, forces[primaryForce].values)
    plt.text(0.5, 350.0, "lsst.sal.MTM1M3.forceActuatorData."+primaryForce, color='g')

    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (nt)")
    plt.subplot(1,2,2)
    if secondaryName is not None:
        plt.title(f"Secondary - {secondaryName} - ID:{id}")
        plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
        plotEnd = plotStart + 14.0 #thisBump[thisBump[secondaryBump]==5]['timestamp'].values[0] + 2.0
        start = Time(plotStart, format='unix_tai', scale='tai')
        end = Time(plotEnd, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        plotStart -= t0
        plotEnd -= t0
        plt.plot(times, forces[secondaryForce].values)
        plt.text(0.5, 350.0, "lsst.sal.MTM1M3.forceActuatorData."+secondaryForce, color='g')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (nt)")
    else:
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
    return

In [ ]:
fig = plt.figure(figsize=(10,5))
await plotBumpTestResults(fig, bumps, 227)

## Given an actuator ID, this plots the cylinder applied forces

In [ ]:
async def plotBumpTestAppliedCylinderForces(fig, bumps, id):
    thisBump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"primaryCylinderForces{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondaryForce = f"secondaryCylinderForces{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    plt.subplots_adjust(wspace=0.3)
    plt.subplot(1,2,1)
    plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
    plotEnd = plotStart + 28.0 #thisBump[thisBump[primaryBump]==5]['timestamp'].values[0] + 2.0
    start = Time(plotStart, format='unix_tai', scale='tai')
    end = Time(plotEnd, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.appliedCylinderForces", [primaryForce, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    plotStart -= t0
    plotEnd -= t0
    plt.title(f"Primary - Z - ID:{id}")
    plt.plot(times, forces[primaryForce].values)
    plt.text(1.0, 340000.0, "lsst.sal.MTM1M3.appliedCylinderForces.\n"+primaryForce, color='g')

    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400000,400000)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force mNt)")
    plt.subplot(1,2,2)
    if secondaryName is not None:
        plt.title(f"Secondary - {secondaryName} - ID:{id}")
        plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
        #plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
        plotEnd = plotStart + 28.0 #thisBump[thisBump[secondaryBump]==5]['timestamp'].values[0] + 2.0
        start = Time(plotStart, format='unix_tai', scale='tai')
        end = Time(plotEnd, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.appliedCylinderForces", [secondaryForce, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        plotStart -= t0
        plotEnd -= t0
        plt.plot(times, forces[secondaryForce].values)
        plt.text(1.0, 340000.0, "lsst.sal.MTM1M3.appliedCylinderForces.\n"+secondaryForce, color='g')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-400000,400000)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (mNt)")
    else:
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
    return

In [ ]:
fig = plt.figure(figsize=(10,5))
await plotBumpTestAppliedCylinderForces(fig, bumps, 227)

## Given an actuator ID, this plots the cylinder measured forces

In [ ]:
async def plotBumpTestCylinderForces(fig, bumps, id):
    thisBump = bumps[bumps['actuatorId']==id]
    index = M1M3FATable.actuatorIDToIndex(id)
    primaryBump = f"primaryTest{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    primaryForce = f"primaryCylinderForce{FATABLE[index][M1M3FATable.FATABLE_ZINDEX]}"
    if FATABLE[index][M1M3FATable.FATABLE_TYPE] == 'DAA':
        secondaryBump = f"secondaryTest{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondaryForce = f"secondaryCylinderForce{FATABLE[index][M1M3FATable.FATABLE_SINDEX]}"
        secondaryName = FATABLE[index][M1M3FATable.FATABLE_ORIENTATION]
    else:
        secondaryName = None

    plt.subplots_adjust(wspace=0.3)
    plt.subplot(1,2,1)
    plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
    plotEnd = plotStart + 28.0 #thisBump[thisBump[primaryBump]==5]['timestamp'].values[0] + 2.0
    start = Time(plotStart, format='unix_tai', scale='tai')
    end = Time(plotEnd, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [primaryForce, 'timestamp'], start.utc, end.utc)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    plotStart -= t0
    plotEnd -= t0
    plt.title(f"Primary - Z - ID:{id}")
    plt.plot(times, forces[primaryForce].values)
    plt.text(1.0, 340.0, "lsst.sal.MTM1M3.forceActuatorData.\n"+primaryForce, color='g')

    plt.xlim(plotStart, plotEnd)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force Nt)")
    plt.subplot(1,2,2)
    if secondaryName is not None:
        plt.title(f"Secondary - {secondaryName} - ID:{id}")
        plotStart = thisBump[thisBump[primaryBump]==2]['timestamp'].values[0] - 1.0
        #plotStart = thisBump[thisBump[secondaryBump]==2]['timestamp'].values[0] - 1.0
        plotEnd = plotStart + 28.0 #thisBump[thisBump[secondaryBump]==5]['timestamp'].values[0] + 2.0
        start = Time(plotStart, format='unix_tai', scale='tai')
        end = Time(plotEnd, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", [secondaryForce, 'timestamp'], start.utc, end.utc)
        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        plotStart -= t0
        plotEnd -= t0
        plt.plot(times, forces[secondaryForce].values)
        plt.text(1.0, 340.0, "lsst.sal.MTM1M3.forceActuatorData.\n"+secondaryForce, color='g')
        plt.xlim(plotStart, plotEnd)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (Nt)")
    else:
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
    return

In [ ]:
fig = plt.figure(figsize=(10,5))
await plotBumpTestCylinderForces(fig, bumps, 227)